# XAS Notebook

The XAS Notebook autoprocessor will run automatically on every scan performed that contains the [NXxas](https://manual.nexusformat.org/classes/applications/NXxas.html#nxxas) sub-entry.

A simple analysis and plot of the data is provided.

Note: [mmg_toolbox](https://github.com/DanPorter/mmg_toolbox) is in active development and likely to change in the future, therefore please don't rely to heavily on the functions at this point. You can see how everything works at the repository. If you have any comments, please do get in touch!

In [ ]:
# Processor requirements
inpath = ""

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt

import hdfmap
print(hdfmap.module_info())

# Set environment variables
os.environ["MPLCONFIGDIR"] = "/tmp/.config/matplotlib"

# Ensure the directories exist
os.makedirs(os.environ["MPLCONFIGDIR"], exist_ok=True)

from mmg_toolbox.env_functions import get_scan_number, replace_scan_number
from mmg_toolbox.spectra_scan import SpectraScan, find_pol_pairs, is_nxxas


In [ ]:
# Load data from NeXus file using HdfMap

nxs_map = hdfmap.create_nexus_map(inpath)

with nxs_map.load_hdf() as nxs:
    def rd(expr, default=''):
        return nxs_map.format_hdf(nxs, expr, default=default)

    # currently accounts for i06-1 and i10-1 metadata
    metadata = {
        "scan": rd('{filename}'),
        "cmd": rd('{(cmd|user_command|scan_command)}'),
        "title": rd('{title}', os.path.basename(inpath)),
        "endstation": rd('{end_station}', 'unknown'),
        "sample": rd('{sample_name}', ''),
        "energy": rd('{np.mean((fastEnergy|pgm_energy|energye|energyh)):.2f} eV'),
        "pol": rd('{polarisation?("lh")}'),
        "height": rd('{(em_y|hfm_y|scm_y?(0)):.2f}', 0),
        "pitch": rd('{(em_pitch|hfm_pitch|m7_pitch?(0)):.2f}', 0),
        "temperature": rd('{(T_sample|sample_temperature|lakeshore336_cryostat|lakeshore336_sample|itc3_device_sensor_temp?(300)):.2f} K'),
        "field": rd('{(field_z|sample_field|magnet_field|ips_demand_field?(0)):.2f} T'),
    }

    print(nxs_map.eval(nxs, '_(fastEnergy|pgm_energy|energye|energyh)'))
    print(nxs_map.eval(nxs, '_(C2|ca62sr|mcs16_data|mcse16_data|mcsh16_data)'))
    print(nxs_map.eval(nxs, '_(C1|ca61sr|mcs17_data|mcse17_data|mcsh17_data)'))
    print(nxs_map.eval(nxs, '_(C3|ca63sr|mcs18_data|mcse18_data|mcsh18_data|mcsd18_data)'))

    energy = nxs_map.eval(nxs, '(fastEnergy|pgm_energy|energye|energyh)')
    monitor = nxs_map.eval(nxs, '(C2|ca62sr|mcs16_data|mcse16_data|mcsh16_data)', default=1.0)
    tey = nxs_map.eval(nxs, '(C1|ca61sr|mcs17_data|mcse17_data|mcsh17_data)', default=np.ones(nxs_map.scannables_shape())) / monitor
    tfy = nxs_map.eval(nxs, '(C3|ca63sr|mcs18_data|mcse18_data|mcsh18_data|mcsd18_data)', default=np.ones(nxs_map.scannables_shape())) / monitor

print('\n'.join(f"{n:12}: {d}" for n, d in metadata.items()))

title = "{endstation} {sample} {scan}\nE = {energy}, pol = {pol}, T = {temperature}, B = {field}".format(**metadata)

In [ ]:
# Load data from NeXus file from NXxas sub-entry
"""
import h5py 
with h5py.File(inpath, 'r') as nxs:
    # first entry
    entry = next(group for path, group in nxs.items() if group.attrs.get('NX_class', b'') == b'NXentry')
    nxxas = next(group for path, group in entry.items() if group.attrs.get('NX_class', b'') == b'NXsubentry' and group.get('definition').asstr()[()] == 'NXxas')
    print(f"NXxas path: \n{nxxas.name}")
    print(f"  {'\n  '.join(nxxas.keys())}")

    monitor = nxxas['monitor']['data'][()] if 'data' in nxxas['monitor'] else 1.0
    data = nxxas['data']  # or 'tey', 'tfy_ft'
    mode = data['mode'][()]
    energy = data['energy'][()]
    absorbed_beam = data['absorbed_beam'][()] / monitor
    tey = nxxas['tey']['absorbed_beam'][()] / monitor
    tfy = nxxas['tfy']['absorbed_beam'][()] / monitor
"""


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
fig.suptitle(title)

ax[0].plot(energy, tey, '-', label=f"{metadata['pol']} TEY")
ax[1].plot(energy, tfy, '-', label=f"{metadata['pol']} TFY")

ax[0].set_xlabel('Energy [eV]')
ax[0].set_ylabel('signal / monitor')
ax[0].legend()

ax[1].set_xlabel('Energy [eV]')
ax[1].set_ylabel('signal / monitor')
ax[1].legend()

# Search for previous NXxas scans
If the previous scan was also a spectra scan, plot this and take the difference

In [ ]:
# Build list of scans
scan_number = get_scan_number(inpath)
scans = []
while len(scans) < 9:
    filename = replace_scan_number(inpath, scan_number)
    scan_number -= 1
    if is_nxxas(filename):
        scans.append(SpectraScan(filename))  # Load scan data
    else:
        break

for scan in scans:
    print(scan, '\n')

# Get polarisations
pols = {scan.pol for scan in scans}
print(f"Polarisations: {pols}")

In [ ]:
# Plot raw spectra
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[12, 6], dpi=100)
for scan in scans:
    scan.tey.plot(ax1)
    scan.tfy.plot(ax2)

ax1.set_xlabel('E [eV]')
ax1.set_ylabel('TEY / monitor')
ax2.set_xlabel('E [eV]')
ax2.set_ylabel('TFY / monitor')
ax1.legend()
ax2.legend()

In [ ]:
# Subtract background and normalise
# background options: flat, norm, linear, curve, exp
# normalisation option: .norm_to_jump(), .norm_to_peak()
rem_bkg = [s.remove_background('flat').norm_to_jump() for s in scans]

# Plot background subtracted, normalised spectra
for scan in rem_bkg:
    # print(scan, '\n')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[12, 6], dpi=100)
    scan.tey.plot(ax1)
    scan.tey.plot_bkg(ax1)
    scan.tfy.plot(ax2)
    scan.tfy.plot_bkg(ax2)

    ax1.set_xlabel('E [eV]')
    ax1.set_ylabel('TEY / monitor')
    ax2.set_xlabel('E [eV]')
    ax2.set_ylabel('TFY / monitor')
    ax1.legend()
    ax2.legend()

In [ ]:
# Find polarisation pairs within the scans
if len(scans) > 1 and len(pols) > 1:
    pairs = find_pol_pairs(*rem_bkg)
    
    for pair in pairs:
        print(pair)
    
        # pair.create_figure()
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[12, 6], dpi=100)
        fig.suptitle(pair.description)
    
        pair.spectra1.tey.plot_parents(ax1)  # background subtracted spectra
        pair.spectra2.tey.plot_parents(ax1)
        pair.tey.plot(ax1)  # XMCD/XMLD
    
        pair.spectra1.tfy.plot_parents(ax2)
        pair.spectra2.tfy.plot_parents(ax2)
        pair.tfy.plot(ax2)
    
        ax1.set_xlabel('E [eV]')
        ax1.set_ylabel('TEY / monitor')
        ax2.set_xlabel('E [eV]')
        ax2.set_ylabel('TFY / monitor')
        ax1.legend()
        ax2.legend()

In [ ]:
# Average each polarisation of all scans
if len(scans) > 1 and len(pols) > 1:
    pol1 = [pair.spectra1 for pair in pairs]
    pol2 = [pair.spectra2 for pair in pairs]
    av_pol1 = sum(pol1[1:], pol1[0])
    av_pol2 = sum(pol2[1:], pol2[0])
    diff = av_pol1 - av_pol2
    print(diff)
    
    # diff.create_figure()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[12, 6], dpi=100)
    fig.suptitle(diff.description)
    
    diff.spectra1.tey.plot_parents(ax1)
    diff.spectra2.tey.plot_parents(ax1)
    diff.tey.plot(ax1)
    
    diff.spectra1.tfy.plot_parents(ax2)
    diff.spectra2.tfy.plot_parents(ax2)
    diff.tfy.plot(ax2)
    
    ax1.set_xlabel('E [eV]')
    ax1.set_ylabel('TEY / monitor')
    ax2.set_xlabel('E [eV]')
    ax2.set_ylabel('TFY / monitor')
    ax1.legend()
    ax2.legend()